In [1]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
import requests
import json
import pandas as pd


In [2]:
pd.set_option('max_colwidth', 150)


In [3]:
key = 'sastry-api-57660e693969.json'


In [4]:
def connect(key):
    """Create a connection to the Google Search Console API and return service object.
    
    Args:
        key (string): Google Search Console JSON client secrets path.
    
    Returns:
        service (object): Google Search Console service object.
    """
    
    scope = ['https://www.googleapis.com/auth/webmasters']
    credentials = service_account.Credentials.from_service_account_file(key, 
                                                                        scopes=scope)
    service = build(
        'webmasters',
        'v3',
        credentials=credentials
    )
    
    return service


In [5]:
def query(service, site_url, payload):
    """Run a query on the Google Search Console API and return a dataframe of results.
    
    Args:
        service (object): Service object from connect()
        site_url (string): URL of Google Search Console property
        payload (dict): API query payload dictionary
    
    Return:
        df (dataframe): Pandas dataframe containing requested data. 
    
    """
    
    response = service.searchanalytics().query(siteUrl=site_url, body=payload).execute()
    
    results = []
    
    for row in response['rows']:    
        data = {}
        
        for i in range(len(payload['dimensions'])):
            data[payload['dimensions'][i]] = row['keys'][i]

        data['clicks'] = row['clicks']
        data['impressions'] = row['impressions']
        data['ctr'] = round(row['ctr'] * 100, 2)
        data['position'] = round(row['position'], 2)        
        results.append(data)
    
    return pd.DataFrame.from_dict(results)


In [6]:
service = connect(key)

payload = {
    'startDate': "2019-01-01",
    'endDate': "2019-12-31",
    'dimensions': ["page","device","query"],
    'rowLimit': 100,
    'startRow': 0
}

print(service)

site_url = "https://gardencentercare.com/"

df = query(service, site_url, payload)
# df.head()


HttpError: <HttpError 403 when requesting https://www.googleapis.com/webmasters/v3/sites/https%3A%2F%2Fgardencentercare.com%2F/searchAnalytics/query?alt=json returned "User does not have sufficient permission for site 'https://gardencentercare.com/'. See also: https://support.google.com/webmasters/answer/2451999.". Details: "[{'message': "User does not have sufficient permission for site 'https://gardencentercare.com/'. See also: https://support.google.com/webmasters/answer/2451999.", 'domain': 'global', 'reason': 'forbidden'}]">

In [ ]:
service.sites().list().uri